In [3]:
# Getting dataset for event management system
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# Simulate data of 50 events with random attributes
num_events = 50
data = {
    'event_id': [f'event_{i+1}' for i in range(num_events)],
    'event_type': [random.choice(['concert', 'conference', 'workshop', 'webinar']) for _ in range(num_events)],
    'event_date': [datetime.now() + timedelta(days=random.randint(1, 365)) for _ in range(num_events)],
    'location': [random.choice(['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix']) for _ in range(num_events)],
    'attendees': [random.randint(50, 500) for _ in range(num_events)],
    'duration_hours': [random.randint(1, 8) for _ in range(num_events)],
    'cost_per_attendee': [round(random.uniform(10.0, 500.0), 2) for _ in range(num_events)],
    'sponsor': [random.choice(['Company A', 'Company B', 'Company C', 'None']) for _ in range(num_events)],
    'feedback_score': [random.uniform(1.0, 5.0) for _ in range(num_events)],
    'last_extraction': [datetime.now() - timedelta(days=random.randint(0, 30)) for _ in range(num_events)],
}
df = pd.DataFrame(data)
df.to_csv('event_dataset.csv', index=False)
df.head()   


,event_id,event_type,event_date,location,attendees,duration_hours,cost_per_attendee,sponsor,feedback_score,last_extraction
0,event_1,workshop,2025-10-30 12:37:56.244200,Phoenix,145,8,400.39,Company C,1.835256,2025-05-31 12:37:56.244200
1,event_2,workshop,2025-09-08 12:37:56.244200,Houston,304,2,39.72,Company C,3.110893,2025-05-22 12:37:56.244200
2,event_3,concert,2026-01-21 12:37:56.244200,Houston,377,3,116.11,Company C,4.199988,2025-05-28 12:37:56.244200
3,event_4,conference,2026-02-11 12:37:56.244200,New York,385,2,482.52,Company A,2.509972,2025-06-01 12:37:56.244200
4,event_5,webinar,2025-12-16 12:37:56.244200,Houston,413,1,412.97,Company A,1.188433,2025-06-13 12:37:56.244200


# FULL Extraction

In [4]:
# Full Extraction
df_full = pd.read_csv("event_dataset.csv", parse_dates=["last_extraction"])
print(f"Extracted {len(df_full)} rows fully.")
df_full.head()

Extracted 50 rows fully.


,event_id,event_type,event_date,location,attendees,duration_hours,cost_per_attendee,sponsor,feedback_score,last_extraction
0,event_1,workshop,2025-10-30 12:37:56.244200,Phoenix,145,8,400.39,Company C,1.835256,2025-05-31 12:37:56.244200
1,event_2,workshop,2025-09-08 12:37:56.244200,Houston,304,2,39.72,Company C,3.110893,2025-05-22 12:37:56.244200
2,event_3,concert,2026-01-21 12:37:56.244200,Houston,377,3,116.11,Company C,4.199988,2025-05-28 12:37:56.244200
3,event_4,conference,2026-02-11 12:37:56.244200,New York,385,2,482.52,Company A,2.509972,2025-06-01 12:37:56.244200
4,event_5,webinar,2025-12-16 12:37:56.244200,Houston,413,1,412.97,Company A,1.188433,2025-06-13 12:37:56.244200


# INCREMENTAL Extraction

In [5]:
# Set initial last extraction time (e.g., halfway through the data range)
with open("last_extraction.txt", "w") as f:
    f.write("2025-06-04 12:00:00") 

In [6]:
# INCREMENTAL EXTRACTION
with open("last_extraction.txt", "r") as f:
    last_extraction = f.read().strip()
df = pd.read_csv("event_dataset.csv", parse_dates=["last_extraction"])
last_extraction_time = pd.to_datetime(last_extraction)
df_incremental = df[df['last_extraction'] > last_extraction_time]
print(f"Extracted {len(df_incremental)}  rows since last check.")
df_incremental.head()

Extracted 15  rows since last check.


,event_id,event_type,event_date,location,attendees,duration_hours,cost_per_attendee,sponsor,feedback_score,last_extraction
4,event_5,webinar,2025-12-16 12:37:56.244200,Houston,413,1,412.97,Company A,1.188433,2025-06-13 12:37:56.244200
6,event_7,workshop,2026-03-28 12:37:56.244200,Los Angeles,130,5,84.42,Company B,2.909601,2025-06-10 12:37:56.244200
7,event_8,workshop,2025-10-16 12:37:56.244200,Los Angeles,262,6,368.00,Company C,1.903332,2025-06-11 12:37:56.244200
10,event_11,workshop,2025-08-11 12:37:56.244200,Houston,433,6,210.97,Company A,4.117752,2025-06-11 12:37:56.244200
11,event_12,conference,2025-12-30 12:37:56.244200,Phoenix,117,1,427.95,Company B,4.993007,2025-06-11 12:37:56.244200


In [7]:
# Successful extraction
# Get the most recent update
new_checkpoint = df['last_extraction'].max()
# Save it
with open("last_extraction.txt", "w") as f:
    f.write(new_checkpoint.isoformat())
print(f"Updated last_extraction.txt to {new_checkpoint}")

Updated last_extraction.txt to 2025-06-14 12:37:56.244200


## Section 4 : Transform Full Data

#### Transformation Requirements

- Cleaning: Handle missing values, remove duplicates. 
- Enrichment: Add calculated columns (e.g., total_price = quantity * unit_price). 
- Structural: Convert data types, standardize date formats. 
- Filtering: Remove irrelevant columns/rows. 
- Categorization: Bin numerical values (e.g., age groups). 


In [40]:
# Transformations for Full Extraction
df_transformed_full = df_full.copy()

# Transformation 1 : Handle missing values and duplicate rows
if df_transformed_full.isnull().values.any():
    print("Missing values found in the dataset. Filling with forward fill method.") 
else:
    print("No missing values found in the dataset.")
    df_transformed_full['event_type'].fillna('unknown', inplace=True)
    df_transformed_full['location'].fillna('unknown', inplace=True)
rows_count_before = len(df_transformed_full)

df_transformed_full.drop_duplicates(inplace=True)
row_count_after = len(df_transformed_full)

df_transformed_full.fillna(method='ffill', inplace=True)
row_count_after = len(df_transformed_full)

print(f"Row count before transformation: {rows_count_before}")
print(f"Row count after transformation: {row_count_after}")    

Missing values found in the dataset. Filling with forward fill method.
Row count before transformation: 50
Row count after transformation: 50


In [31]:
# Transformation 2: Enrichment - Total cost calculation
if 'attendees' in df_transformed_full.columns and 'cost_per_attendee' in df_transformed_full.columns:
    df_transformed_full['total_cost'] = round(df_transformed_full['attendees'] * df_transformed_full['cost_per_attendee'], 2)  
else:
    print("Columns 'attendees' or 'cost_per_attendee' not found in DataFrame for total cost calculation.")

print("Transformed DataFrame with total cost:")
print(df_transformed_full[['attendees', 'cost_per_attendee', 'total_cost']])   

Transformed DataFrame with total cost:
    attendees  cost_per_attendee  total_cost
0         145             400.39    58056.55
1         304              39.72    12074.88
2         377             116.11    43773.47
3         385             482.52   185770.20
4         413             412.97   170556.61
5         288             110.84    31921.92
6         130              84.42    10974.60
7         262             368.00    96416.00
8         406              41.58    16881.48
9         374             121.46    45426.04
10        433             210.97    91350.01
11        117             427.95    50070.15
12         75              23.79     1784.25
13        147             399.74    58761.78
14         66             360.08    23765.28
15        196             455.94    89364.24
16        399             210.63    84041.37
17        455             462.53   210451.15
18         89             450.41    40086.49
19        458             212.40    97279.20
20        327   

In [32]:
# Transformation 3: Structural - Convert event_date to string format
if 'event_date' in df_transformed_full.columns: 
    df_transformed_full['event_date'] = df_transformed_full['event_date'].astype(str)
else:
    print("Column 'event_date' not found in DataFrame for conversion to string format.")

print("Transformed DataFrame with event_date as string:")
print(df_transformed_full[['event_date']])


Transformed DataFrame with event_date as string:
    event_date
0   2025-10-30
1   2025-09-08
2   2026-01-21
3   2026-02-11
4   2025-12-16
5   2026-01-09
6   2026-03-28
7   2025-10-16
8   2025-07-11
9   2025-11-26
10  2025-08-11
11  2025-12-30
12  2025-07-25
13  2025-07-13
14  2026-05-07
15  2025-12-17
16  2026-05-22
17  2026-04-13
18  2025-06-21
19  2025-10-13
20  2026-04-27
21  2026-02-22
22  2026-03-23
23  2026-06-11
24  2025-10-11
25  2025-10-12
26  2025-07-27
27  2026-04-02
28  2025-08-21
29  2026-02-23
30  2026-02-04
31  2026-01-30
32  2026-01-02
33  2026-02-04
34  2025-08-04
35  2025-07-07
36  2026-03-31
37  2025-09-10
38  2026-02-08
39  2025-11-13
40  2026-02-05
41  2025-09-03
42  2025-12-24
43  2025-12-30
44  2026-04-02
45  2026-03-19
46  2026-02-16
47  2026-01-12
48  2025-12-19
49  2025-11-03


In [33]:
# sample of transformed full data
df_transformed_sample = df_transformed_full.sample(5)
print("Sample of transformed full data:")
print(df_transformed_sample)

Sample of transformed data:
    event_id event_type  event_date location  attendees  duration_hours  \
39  event_40    webinar  2025-11-13  Houston        310               1   
1    event_2   workshop  2025-09-08  Houston        304               2   
15  event_16    webinar  2025-12-17  Houston        196               1   
27  event_28   workshop  2026-04-02  Phoenix         83               1   
35  event_36    webinar  2025-07-07  Houston        317               6   

    cost_per_attendee    sponsor  feedback_score            last_extraction  \
39             396.60  Company A        3.482855 2025-05-19 12:37:56.244200   
1               39.72  Company C        3.110893 2025-05-22 12:37:56.244200   
15             455.94  Company C        1.764903 2025-06-03 12:37:56.244200   
27             367.70  Company A        1.906805 2025-06-12 12:37:56.244200   
35             485.35  Company A        4.871442 2025-05-27 12:37:56.244200   

    total_cost  
39   122946.00  
1     12074.

In [53]:
# save transformed full dataset
df_transformed_full.to_csv('transformed_full.csv', index=False)
df_transformed_full.head()

,event_id,event_type,event_date,location,attendees,duration_hours,cost_per_attendee,sponsor,feedback_score,last_extraction
0,event_1,workshop,2025-10-30 12:37:56.244200,Phoenix,145,8,400.39,Company C,1.835256,2025-05-31 12:37:56.244200
1,event_2,workshop,2025-09-08 12:37:56.244200,Houston,304,2,39.72,Company C,3.110893,2025-05-22 12:37:56.244200
2,event_3,concert,2026-01-21 12:37:56.244200,Houston,377,3,116.11,Company C,4.199988,2025-05-28 12:37:56.244200
3,event_4,conference,2026-02-11 12:37:56.244200,New York,385,2,482.52,Company A,2.509972,2025-06-01 12:37:56.244200
4,event_5,webinar,2025-12-16 12:37:56.244200,Houston,413,1,412.97,Company A,1.188433,2025-06-13 12:37:56.244200


## Section 5 : Transform Incremental Data


In [42]:
# Transformations for Incremental Extraction
df_transformed_incremental = df_incremental.copy()

# Transformation 1: Handle missing values and duplicate rows
if df_transformed_incremental.isnull().values.any():
    print("Missing values found in the dataset. Filling with forward fill method.")
else:
    print("No missing values found in the dataset.")
    df_transformed_incremental['event_type'].fillna('unknown', inplace=True)
    df_transformed_incremental['location'].fillna('unknown', inplace=True)
rows_count_before = len(df_transformed_incremental)

df_transformed_incremental.drop_duplicates(inplace=True)
row_count_after = len(df_transformed_incremental)

df_transformed_incremental.fillna(method='ffill', inplace=True)
row_count_after = len(df_transformed_incremental)

print(f"Row count before transformation: {rows_count_before}")
print(f"Row count after transformation: {row_count_after}")

Missing values found in the dataset. Filling with forward fill method.
Row count before transformation: 15
Row count after transformation: 15


In [44]:
# Transformation 2: Enrichment - Total cost calculation
df_transformed_incremental['total_cost'] = round(df_transformed_incremental['attendees'] * df_transformed_incremental['cost_per_attendee'], 2)
if 'attendees' in df_transformed_incremental.columns and 'cost_per_attendee' in df_transformed_incremental.columns:
    df_transformed_incremental['total_cost'] = round(df_transformed_incremental['attendees'] * df_transformed_incremental['cost_per_attendee'], 2)
else:
    print("Columns 'attendees' or 'cost_per_attendee' not found in DataFrame for total cost calculation.")

print("Transformed DataFrame with total cost:")
print(df_transformed_incremental[['attendees', 'cost_per_attendee', 'total_cost']])


Transformed DataFrame with total cost:
    attendees  cost_per_attendee  total_cost
4         413             412.97   170556.61
6         130              84.42    10974.60
7         262             368.00    96416.00
10        433             210.97    91350.01
11        117             427.95    50070.15
20        327             453.56   148314.12
21        161             333.31    53662.91
22        327             125.67    41094.09
25        311              50.14    15593.54
27         83             367.70    30519.10
28        494             477.76   236013.44
40        297             458.39   136141.83
42        237             130.94    31032.78
45        328              93.38    30628.64
46        458             270.43   123856.94


In [47]:
# Transformation 3: Structural - Convert event_date to datetime string format
if 'event_date' in df_transformed_incremental.columns:
    df_transformed_incremental['event_date'] = pd.to_datetime(df_transformed_incremental['event_date']).dt.strftime('%Y-%m-%d')
else:
    print("Column 'event_date' not found in DataFrame for conversion to datetime string format.")
print("Transformed DataFrame with event_date as string:")
print(df_transformed_incremental[['event_date']])

Transformed DataFrame with event_date as string:
    event_date
4   2025-12-16
6   2026-03-28
7   2025-10-16
10  2025-08-11
11  2025-12-30
20  2026-04-27
21  2026-02-22
22  2026-03-23
25  2025-10-12
27  2026-04-02
28  2025-08-21
40  2026-02-05
42  2025-12-24
45  2026-03-19
46  2026-02-16


In [48]:
# sample of transformed incremental data
df_transformed_incremental_sample = df_transformed_incremental.sample(5)
print("Sample of transformed incremental data:")
print(df_transformed_incremental_sample)

Sample of transformed incremental data:
    event_id event_type  event_date     location  attendees  duration_hours  \
45  event_46    webinar  2026-03-19      Houston        328               3   
20  event_21    concert  2026-04-27      Phoenix        327               1   
6    event_7   workshop  2026-03-28  Los Angeles        130               5   
10  event_11   workshop  2025-08-11      Houston        433               6   
22  event_23   workshop  2026-03-23  Los Angeles        327               1   

    cost_per_attendee    sponsor  feedback_score            last_extraction  \
45              93.38  Company B        2.510312 2025-06-07 12:37:56.244200   
20             453.56  Company B        2.690193 2025-06-04 12:37:56.244200   
6               84.42  Company B        2.909601 2025-06-10 12:37:56.244200   
10             210.97  Company A        4.117752 2025-06-11 12:37:56.244200   
22             125.67  Company A        4.594836 2025-06-12 12:37:56.244200   

    total_

In [51]:
# save transformed incremental dataset
df_transformed_incremental.to_csv('transformed_incremental_data.csv', index=False)
df_transformed_incremental.head()

,event_id,event_type,event_date,location,attendees,duration_hours,cost_per_attendee,sponsor,feedback_score,last_extraction,total_cost
4,event_5,webinar,2025-12-16,Houston,413,1,412.97,Company A,1.188433,2025-06-13 12:37:56.244200,170556.61
6,event_7,workshop,2026-03-28,Los Angeles,130,5,84.42,Company B,2.909601,2025-06-10 12:37:56.244200,10974.60
7,event_8,workshop,2025-10-16,Los Angeles,262,6,368.00,Company C,1.903332,2025-06-11 12:37:56.244200,96416.00
10,event_11,workshop,2025-08-11,Houston,433,6,210.97,Company A,4.117752,2025-06-11 12:37:56.244200,91350.01
11,event_12,conference,2025-12-30,Phoenix,117,1,427.95,Company B,4.993007,2025-06-11 12:37:56.244200,50070.15
